In [1]:
import os
from pathlib import Path
import pandas as pd
import QuantLib as ql
file_dir = os.path.join(str(Path().resolve()),'historical_data_collection','historical_av_collection')
files = os.listdir(file_dir)
filename = [f for f in files if f.endswith('.h5')][0]

file = os.path.join(file_dir,filename)
with pd.HDFStore(file) as store:
    df = store["/date_2022_10_20/heston_calibration/calibration_results"]
eg = df.iloc[0]
eg

strike_price             335.0
w                         call
market_price              34.6
volatility             0.33141
days_to_maturity            34
calculation_date    2022-10-20
spot_price              365.41
risk_free_rate            0.04
dividend_rate              0.0
theta                  0.17771
kappa                 0.412367
eta                   0.785592
rho                  -0.582856
v0                     0.08079
moneyness             0.090776
black_scholes        35.227613
heston_price          34.77996
error                -0.447652
Name: 3420, dtype: object

In [2]:
calculation_date = ql.Date.todaysDate()

r = 0.04
g = 0.0
s = 365.41
k = 335.0
w = 'call'

expiration_date = calculation_date + ql.Period(34,ql.Days)

kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079 


past_fixings = 0
rng = "pseudorandom" # could use "lowdiscrepancy"
numPaths = 100000
geometricRunningAccumulator = 1.0
fixing_dates = [expiration_date]

riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, r, ql.Actual365Fixed()))
dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, g, ql.Actual365Fixed()))

hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
hestonModel = ql.HestonModel(hestonProcess)
vanillaPayoff = ql.PlainVanillaPayoff(ql.Option.Call, k)
europeanExercise = ql.EuropeanExercise(expiration_date)

In [3]:
vanilla_engine = ql.AnalyticHestonEngine(hestonModel)
vanilla = ql.VanillaOption(vanillaPayoff, europeanExercise)
vanilla.setPricingEngine(vanilla_engine)
vanilla.NPV()

34.779970977560126

In [4]:
asian_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)

geometricAverage = ql.Average().Geometric

discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
	geometricAverage, geometricRunningAccumulator, past_fixings, fixing_dates, vanillaPayoff, europeanExercise
)
discreteGeometricAsianOption.setPricingEngine(asian_engine)
discreteGeometricAsianOption.NPV()

34.70413563954013